# ***YOLOv12 + R-ELAN + BiFPN***

# YOLOv12n R-ELAN + BiFPN (Advanced Multi-Scale Fusion)  
**Versi paling canggih & terbaik secara teori untuk object detection 2024–2025**

Keunggulan BiFPN:
- Weighted feature fusion (learnable weights) → lebih pintar dari Concat biasa  
- Bidirectional path (top-down + bottom-up) → informasi mengalir maksimal  
- Lebih efisien dari PANet + lebih akurat dari FPN  
- Sudah terbukti di EfficientDet (Google) → mAP jauh lebih tinggi pada dataset kompleks seperti dental caries

Ini adalah varian yang **harus kamu coba terakhir** — biasanya jadi yang terbaik.

## 1. Install Library & Patch Custom BiFPN_Concat2 Module

In [1]:
!pip install -q ultralytics optuna pandas roboflow PyYAML --upgrade

import optuna
from ultralytics import YOLO
from ultralytics.nn import tasks
from pathlib import Path
from roboflow import Roboflow
import yaml
import json
import pandas as pd
import torch
import torch.nn as nn
import inspect
import re

# ==================== CUSTOM BiFPN MODULE ====================
class BiFPN_Concat2(nn.Module):
    def __init__(self, dimension=1):
        super().__init__()
        self.d = dimension
        self.w = nn.Parameter(torch.ones(2, dtype=torch.float32), requires_grad=True)
        self.epsilon = 0.0001

    def forward(self, x):
        w = nn.functional.relu(self.w)
        weight = w / (torch.sum(w, dim=0) + self.epsilon)
        return torch.cat([weight[0] * x[0], weight[1] * x[1]], dim=self.d)

# Patch ke Ultralytics
tasks.BiFPN_Concat2 = BiFPN_Concat2

# Modifikasi parse_model agar mengenali BiFPN_Concat2
source = inspect.getsource(tasks.parse_model)
pattern = r'(\s*)else:\s*\n(\s*)c2\s*=\s*ch\s*\[f\](?=\s|$)'
replacement = r'\1elif m is BiFPN_Concat2:\n\2c2 = sum(ch[x] for x in f)\n\1else:\n\2c2 = ch[f]'
modified_source = re.sub(pattern, replacement, source, flags=re.MULTILINE)
exec(modified_source, tasks.__dict__)

print("BiFPN_Concat2 berhasil di-patch ke Ultralytics!")

BiFPN_Concat2 berhasil di-patch ke Ultralytics!


## 2. Pengaturan Global

In [2]:
# ====================== GLOBAL SETTINGS ======================
VARIASI_NAME     = "yolo12m_r-elan_bifpn"
VERSION_NUM      = 13
EPOCHS_SEARCH    = 20
EPOCHS_FINAL     = 500
BATCH_CANDIDATES = [16, 32, 48, 64, 96, 128]

OPTUNA_ROOT      = Path("runs/optuna")
VARIASI_DIR      = OPTUNA_ROOT / VARIASI_NAME
VARIASI_DIR.mkdir(parents=True, exist_ok=True)

DB_PATH     = VARIASI_DIR / "optuna_study.db"
JSON_PATH   = VARIASI_DIR / "best_hyperparameters.json"
CSV_PATH    = VARIASI_DIR / "optuna_trials_log.csv"
YAML_PATH   = VARIASI_DIR / "yolo12m_r-elan_bifpn.yaml"

## 3. Custom YAML: R-ELAN Backbone + Full BiFPN Neck  
Menggunakan 4 weighted fusion (BiFPN_Concat2) → top-down + bottom-up path.

In [3]:
yaml_content = """
nc: 7
scales:
  n: [0.50, 0.25, 1024]
  s: [0.50, 0.50, 1024]
  m: [0.50, 1.00, 512]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.50, 512]

backbone:
  - [-1, 1, Conv, [64, 3, 2]]       # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]      # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, Conv, [256, 3, 2]]      # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, Conv, [512, 3, 2]]      # 5-P4/16
  - [-1, 4, A2C2f, [512, True, 4]]  # R-ELAN
  - [-1, 1, Conv, [1024, 3, 2]]     # 7-P5/32
  - [-1, 4, A2C2f, [1024, True, 1]] # 8

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]      # 9
  - [[-1, 6], 1, BiFPN_Concat2, [1]]                # 10 ← weighted fusion
  - [-1, 2, A2C2f, [512, False, -1]]                # 11
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]      # 12
  - [[-1, 4], 1, BiFPN_Concat2, [1]]                # 13 ← weighted
  - [-1, 2, A2C2f, [256, False, -1]]                # 14
  - [-1, 1, Conv, [256, 3, 2]]                      # 15 downsample
  - [[-1, 11], 1, BiFPN_Concat2, [1]]               # 16 ← bottom-up path
  - [-1, 2, A2C2f, [512, False, -1]]                # 17
  - [-1, 1, Conv, [512, 3, 2]]                      # 18 downsample
  - [[-1, 8], 1, BiFPN_Concat2, [1]]                # 19 ← bottom-up path
  - [-1, 2, C3k2, [1024, True]]                     # 20
  - [[14, 17, 20], 1, Detect, [nc]]                 # 21 Detect
"""

with open(YAML_PATH, "w") as f:
    yaml.dump(yaml.safe_load(yaml_content), f, sort_keys=False)

print(f"Custom BiFPN YAML berhasil dibuat → {YAML_PATH}")

Custom BiFPN YAML berhasil dibuat → runs/optuna/yolo12m_r-elan_bifpn/yolo12m_r-elan_bifpn.yaml


## 4. Download Dataset Dental Caries

In [4]:
API_KEY     = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE   = "dentalogic8"
PROJECT_ID  = "dental-caries-7kttb"

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"dental-caries-{VERSION_NUM}")
DATA_YAML   = DATASET_DIR / "data.yaml"

if not DATA_YAML.exists():
    raise FileNotFoundError("data.yaml tidak ditemukan!")

print(f"Dataset v{VERSION_NUM} siap → {DATA_YAML}")

loading Roboflow workspace...
loading Roboflow project...
Dataset v13 siap → dental-caries-13/data.yaml


## 5. Inisialisasi CSV + Combined Score (Medis Dental)
Bobot:
- **50%** mAP50-95 → akurasi keseluruhan  
- **30%** Recall → kritis (tidak boleh ada karies terlewat)  
- **20%** Precision → kontrol false positive

In [5]:
def init_csv():
    if not CSV_PATH.exists():
        pd.DataFrame(columns=[
            "Trial","Batch","Epochs_Completed","lr0","lrf","weight_decay",
            "box","cls","dfl","mAP@50","mAP@50-95","Precision","Recall",
            "Combined_Score","Status"
        ]).to_csv(CSV_PATH, index=False)
        print(f"CSV log dibuat → {CSV_PATH}")
init_csv()

def get_score(m):
    map5095 = m.get("metrics/mAP50-95(B)", 0.0)
    recall  = m.get("metrics/recall(B)", 0.0)
    prec    = m.get("metrics/precision(B)", 0.0)
    return 0.5*map5095 + 0.3*recall + 0.2*prec, map5095, recall, prec

CSV log dibuat → runs/optuna/yolo12m_r-elan_bifpn/optuna_trials_log.csv


## 6. Optuna Hyperparameter Search – R-ELAN + BiFPN

In [ ]:
study = optuna.create_study(
    study_name=f"dental_caries_{VARIASI_NAME}",
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42),
    storage=f"sqlite:///{DB_PATH}",
    load_if_exists=True
)

def objective(trial):
    batch = trial.suggest_categorical("batch", BATCH_CANDIDATES)
    lr0   = trial.suggest_float("lr0", 1e-4, 0.1, log=True)
    lrf   = trial.suggest_float("lrf", 0.005, 0.2, log=True)
    wd    = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    box   = trial.suggest_float("box", 7.5, 25.0)
    cls   = trial.suggest_float("cls", 0.3, 1.5)
    dfl   = trial.suggest_float("dfl", 0.5, 4.0)

    model = YOLO(str(YAML_PATH))

    try:
        results = model.train(
            data=str(DATA_YAML),
            imgsz=640,
            epochs=EPOCHS_SEARCH,
            batch=batch,
            device=4,
            project=str(VARIASI_DIR),
            name=f"trial_{trial.number:04d}",
            exist_ok=True,
            pretrained=False,
            optimizer="AdamW",
            lr0=lr0, lrf=lrf, weight_decay=wd,
            box=box, cls=cls, dfl=dfl,
            patience=10,
            deterministic=True,
            cache="disk",
            verbose=False,
            plots=False,
            close_mosaic=5,
            amp=True
        )
        m = results.results_dict
        score, map5095, recall, prec = get_score(m)
        map50 = m.get("metrics/mAP50(B)", 0.0)
        status = "Completed"
    except Exception as e:
        print(f"Trial {trial.number} GAGAL → {str(e)[:100]}")
        score = map5095 = recall = prec = map50 = 0.0
        status = "Failed"

    pd.DataFrame([{
        "Trial": trial.number, "Batch": batch, "Epochs_Completed": EPOCHS_SEARCH,
        "lr0": round(lr0,8), "lrf": round(lrf,6), "weight_decay": wd,
        "box": round(box,4), "cls": round(cls,4), "dfl": round(dfl,4),
        "mAP@50": round(map50,5), "mAP@50-95": round(map5095,5),
        "Precision": round(prec,5), "Recall": round(recall,5),
        "Combined_Score": round(score,6), "Status": status
    }]).to_csv(CSV_PATH, mode='a', header=False, index=False)

    print(f"Trial {trial.number:3d} │ B{batch:3d} │ mAP50-95 {map5095:.5f} │ Score {score:.6f}")
    return score

print(f"\n{'='*95}")
print(f"OPTUNA SEARCH: {VARIASI_NAME.upper()} (R-ELAN + BiFPN)")
print(f"YAML        : {YAML_PATH.name}")
print(f"Trial epochs: {EPOCHS_SEARCH} │ Final epochs: {EPOCHS_FINAL}")
print(f"{'='*95}\n")

study.optimize(objective, n_trials=20)

[I 2025-11-22 14:49:48,300] A new study created in RDB with name: dental_caries_yolo12m_r-elan_bifpn



OPTUNA SEARCH: YOLO12M_R-ELAN_BIFPN (R-ELAN + BiFPN)
YAML        : yolo12m_r-elan_bifpn.yaml
Trial epochs: 20 │ Final epochs: 500

Ultralytics 8.3.230 🚀 Python-3.10.12 torch-2.9.1+cu128 CUDA:4 (NVIDIA H200, 143167MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=19.891270111430796, cache=disk, cfg=None, classes=None, close_mosaic=5, cls=0.32470139315496294, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dental-caries-13/data.yaml, degrees=0.0, deterministic=True, device=4, dfl=3.89468448256698, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00014936568554617635, lrf=0.12207764786954153, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod

## 7. Simpan Best Hyperparameter

In [ ]:
with open(JSON_PATH, "w") as f:
    json.dump(study.best_params, f, indent=4)

print(f"\nOPTIMASI SELESAI! Best Score: {study.best_value:.6f}")
print(f"Best params → {JSON_PATH}")
print(f"Log lengkap → {CSV_PATH}")

## 8. Final Training → R-ELAN + BiFPN

In [ ]:
!pip install -q ultralytics --upgrade
from ultralytics import YOLO
from ultralytics.nn import tasks
import inspect, re, torch, torch.nn as nn
import json

# Patch ulang BiFPN (wajib di cell terpisah)
class BiFPN_Concat2(nn.Module):
    def __init__(self, dimension=1):
        super().__init__()
        self.d = dimension
        self.w = nn.Parameter(torch.ones(2, dtype=torch.float32), requires_grad=True)
        self.epsilon = 0.0001
    def forward(self, x):
        w = nn.functional.relu(self.w)
        weight = w / (torch.sum(w, dim=0) + self.epsilon)
        return torch.cat([weight[0] * x[0], weight[1] * x[1]], dim=self.d)

tasks.BiFPN_Concat2 = BiFPN_Concat2
source = inspect.getsource(tasks.parse_model)
pattern = r'(\s*)else:\s*\n(\s*)c2\s*=\s*ch\s*\[f\](?=\s|$)'
replacement = r'\1elif m is BiFPN_Concat2:\n\2c2 = sum(ch[x] for x in f)\n\1else:\n\2c2 = ch[f]'
exec(re.sub(pattern, replacement, source, flags=re.MULTILINE), tasks.__dict__)

# Load best params
with open(JSON_PATH) as f:
    best = json.load(f)

print("Best Hyperparameters (R-ELAN + BiFPN):")
for k, v in best.items():
    print(f"  {k:12} → {v}")

model = YOLO(str(YAML_PATH))

model.train(
    data=str(DATA_YAML),
    imgsz=640,
    epochs=EPOCHS_FINAL,
    batch=best["batch"],
    device=0,
    project="runs/final",
    name=f"{VARIASI_NAME}_FINAL",
    exist_ok=True,
    pretrained=False,
    optimizer="AdamW",
    patience=50,
    cache="disk",
    plots=True,
    save=True,
    close_mosaic=50,
    lr0=best["lr0"],
    lrf=best["lrf"],
    weight_decay=best["weight_decay"],
    box=best["box"],
    cls=best["cls"],
    dfl=best["dfl"],
)

print(f"\nFINAL TRAINING BiFPN SELESAI! Model terbaik:")
print(f"runs/final/{VARIASI_NAME}_FINAL/weights/best.pt")